## 10-fold Cross Validation on the Stanik dataset

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 4.8 MB/s 
     |████████████████████████████████| 1.9 MB 69.1 MB/s 
     |████████████████████████████████| 10.3 MB 53.0 MB/s 
     |████████████████████████████████| 5.5 MB 77.0 MB/s 
     |████████████████████████████████| 451 kB 81.5 MB/s 
     |████████████████████████████████| 1.3 MB 76.3 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 7.6 MB 70.8 MB/s 
     |████████████████████████████████| 182 kB 82.0 MB/s 
     |████████████████████████████████| 168 kB 77.7 MB/s 
     |████████████████████████████████| 182 kB 76.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 168 kB 84.7 MB/s 
     |████████████████████████████████| 166 kB 85.8 MB/s 
     |████████████████████████████████| 166 kB 85.6 MB/s 
     |█████████████████████████████

In [ ]:
# Import downsampled dataset
df = pd.read_excel('DATASET_downsampled_test3.xlsx')

df = df.drop(columns = ['Unnamed: 0'])

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       2592 non-null   object
 1   kano_labels  2592 non-null   object
 2   labels       2592 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 60.9+ KB


In [ ]:
# Show the distribution of the labels
print(df['labels'].value_counts())

1    648
3    648
2    648
0    648
Name: labels, dtype: int64


In [ ]:
# Rename the columns so that simple transformers can handle them
train_data = df[['review','labels']]
train_data.columns = ['text', 'labels']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2592 non-null   object
 1   labels  2592 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 40.6+ KB


In [ ]:
# Model implementation and k-fold cross validation with Simpletransformers

# Wrapper function to calculate f1-score with additional parameters
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = None)

# Wrapper function to calculate recall with additional parameters
def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = None)

# Wrapper function to calculate precision with additional parameters
def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = None)

# K-fold cross validation with 10 folds
n = 10
kf = KFold(n_splits=n, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc = []
resultsf1 = []
resultsPrecision = []
resultsRecall = []

model_args = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create the ClassificationModel
model = ClassificationModel(
    'rembert',
    'google/rembert',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf.split(train_data):
  train_df = train_data.iloc[train_index]
  val_df = train_data.iloc[val_index]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)
  print(result)
  
  # Add scores to the respective lists
  resultsAcc.append(result['acc'])
  resultsf1.append(result['f1'])
  resultsPrecision.append(result['precision'])
  resultsRecall.append(result['recall'])

  print("results accuracy",resultsAcc)
  print(f"Mean-Precision: {sum(resultsAcc) / len(resultsAcc)}")
  print('results f1: ', resultsf1)
  print('results precision: ', resultsPrecision)
  print('results recall: ', resultsRecall)

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

Some weights of RemBertForSequenceClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

...

{'mcc': 0.9384475162085935, 'f1': array([0.96      , 0.96969697, 0.96610169, 0.92307692]), 'acc': 0.9536679536679536, 'recall': array([0.96774194, 0.98461538, 0.98275862, 0.89189189]), 'precision': array([0.95238095, 0.95522388, 0.95      , 0.95652174]), 'eval_loss': 0.18968374078924005}
results accuracy [0.7307692307692307, 0.7961538461538461, 0.8803088803088803, 0.8416988416988417, 0.8648648648648649, 0.8957528957528957, 0.8957528957528957, 0.9227799227799228, 0.9536679536679536]
Mean-Precision: 0.8646388146388146
results f1:  [array([0.86330935, 0.69354839, 0.83116883, 0.44660194]), array([0.90076336, 0.78106509, 0.81904762, 0.67826087]), array([0.92063492, 0.83333333, 0.90140845, 0.85915493]), array([0.92413793, 0.8       , 0.896     , 0.72222222]), array([0.91970803, 0.8503937 , 0.85454545, 0.83333333]), array([0.95833333, 0.88      , 0.92307692, 0.81818182]), array([0.95364238, 0.859375  , 0.944     , 0.80701754]), array([0.92929293, 0.921875  , 0.96052632, 0.87769784]), array([0

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/292 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'mcc': 0.9433865148285822, 'f1': array([0.97520661, 0.95104895, 0.97058824, 0.93220339]), 'acc': 0.9575289575289575, 'recall': array([0.96721311, 0.97142857, 0.95652174, 0.93220339]), 'precision': array([0.98333333, 0.93150685, 0.98507463, 0.93220339]), 'eval_loss': 0.2360415675423362}
results accuracy [0.7307692307692307, 0.7961538461538461, 0.8803088803088803, 0.8416988416988417, 0.8648648648648649, 0.8957528957528957, 0.8957528957528957, 0.9227799227799228, 0.9536679536679536, 0.9575289575289575]
Mean-Precision: 0.8739278289278289
results f1:  [array([0.86330935, 0.69354839, 0.83116883, 0.44660194]), array([0.90076336, 0.78106509, 0.81904762, 0.67826087]), array([0.92063492, 0.83333333, 0.90140845, 0.85915493]), array([0.92413793, 0.8       , 0.896     , 0.72222222]), array([0.91970803, 0.8503937 , 0.85454545, 0.83333333]), array([0.95833333, 0.88      , 0.92307692, 0.81818182]), array([0.95364238, 0.859375  , 0.944     , 0.80701754]), array([0.92929293, 0.921875  , 0.96052632, 0.8

In [ ]:
print("results accuracy",resultsAcc)
print(f"Mean-accuracy: {sum(resultsAcc) / len(resultsAcc)}")
print('results f1: ', resultsf1)
print('results precision: ', resultsPrecision)
print('results recall: ', resultsRecall)

results accuracy [0.7307692307692307, 0.7961538461538461, 0.8803088803088803, 0.8416988416988417, 0.8648648648648649, 0.8957528957528957, 0.8957528957528957, 0.9227799227799228, 0.9536679536679536, 0.9575289575289575]
Mean-accuracy: 0.8739278289278289
results f1:  [array([0.86330935, 0.69354839, 0.83116883, 0.44660194]), array([0.90076336, 0.78106509, 0.81904762, 0.67826087]), array([0.92063492, 0.83333333, 0.90140845, 0.85915493]), array([0.92413793, 0.8       , 0.896     , 0.72222222]), array([0.91970803, 0.8503937 , 0.85454545, 0.83333333]), array([0.95833333, 0.88      , 0.92307692, 0.81818182]), array([0.95364238, 0.859375  , 0.944     , 0.80701754]), array([0.92929293, 0.921875  , 0.96052632, 0.87769784]), array([0.96      , 0.96969697, 0.96610169, 0.92307692]), array([0.97520661, 0.95104895, 0.97058824, 0.93220339])]
results precision:  [array([0.83333333, 0.74137931, 0.79012346, 0.46938776]), array([0.86764706, 0.68041237, 0.93478261, 0.79591837]), array([0.92063492, 0.78947368

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

def PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1):

  np_resultsPrecision = np.array(resultsPrecision)
  np_resultsRecall = np.array(resultsRecall)
  np_resultsf1  = np.array(resultsf1)

  # Calculate the mean precision for each label
  df_resultsPrecision = pd.DataFrame(np_resultsPrecision, columns=list('BDIP'))
  precision_b_mean = df_resultsPrecision['B'].mean()
  precision_d_mean = df_resultsPrecision['D'].mean()
  precision_i_mean = df_resultsPrecision['I'].mean()
  precision_p_mean = df_resultsPrecision['P'].mean()

  # Calculate the mean recall for each label
  df_resultsRecall = pd.DataFrame(np_resultsRecall, columns=list('BDIP'))
  recall_b_mean = df_resultsRecall['B'].mean()
  recall_d_mean = df_resultsRecall['D'].mean()
  recall_i_mean = df_resultsRecall['I'].mean()
  recall_p_mean = df_resultsRecall['P'].mean()

  # Calculate the mean f1-score for each label
  df_resultsf1 = pd.DataFrame(np_resultsf1, columns=list('BDIP'))
  f1_b_mean = df_resultsf1['B'].mean()
  f1_d_mean = df_resultsf1['D'].mean()
  f1_i_mean = df_resultsf1['I'].mean()
  f1_p_mean = df_resultsf1['P'].mean()

  # Create dataframe with summarized results
  metrics = [{'B': precision_b_mean, 'D': precision_d_mean, 'I': precision_i_mean, 'P':precision_p_mean},
          {'B': recall_b_mean, 'D': recall_d_mean, 'I': recall_i_mean, 'P': recall_p_mean},
          {'B': f1_b_mean, 'D': f1_d_mean, 'I': f1_i_mean, 'P': f1_p_mean} ]
  train_df_metrics = pd.DataFrame(metrics, index=['Average precision',
                                'Average recall',
                                'Average F1-Score'])
  return train_df_metrics

PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1)

,B,D,I,P
Average precision,0.915801,0.842813,0.919356,0.814509
Average recall,0.946089,0.871481,0.899178,0.769355
Average F1-Score,0.930503,0.854034,0.906646,0.789775


## Testing on the Brunotte dataset

In [ ]:
# Import of 'Trainingskorpus_Final', which gives the final labeling
testset1 = pd.read_excel('Trainingskorpus_Final.xlsx')

testset1 = testset1.drop(columns = ['Unnamed: 0'])
testset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
dtypes: object(2)
memory usage: 25.5+ KB


In [ ]:
# Show the distribution of the labels
print(testset1['kano_labels'].value_counts())

B    1102
P     395
D      95
I      30
Name: kano_labels, dtype: int64


In [ ]:
# Import of 'Trainingskorpus_InitialLabels', which gives the initial labeling of two sometimes differing labels
testset2 = pd.read_excel('Trainingskorpus_InitialLabels.xlsx')

testset2 = testset2.drop(columns = ['Unnamed: 0'])
testset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   1622 non-null   object
 1   label 1  1622 non-null   object
 2   label 2  1622 non-null   object
dtypes: object(3)
memory usage: 38.1+ KB


In [ ]:
# Merge the two sets so we have the final labels as well as the initial labels in the testset
testset = testset1.merge(testset2, left_on='review', right_on='review')
testset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622 entries, 0 to 1621
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
 2   label 1      1622 non-null   object
 3   label 2      1622 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB


In [ ]:
# Add a column 'equal' which is True if the two initial labels are equal and false otherwise
equal = [None]*np.size(testset['review'])

# Add a column 'equal01' which stores binary values (1 if the two initial labels are equal, 0 otherwise) to calculate the phi-coefficient
equal01 = [None]*np.size(testset['review'])

# Add a column 'labeled_correctly01' which stores binary values (1 if the model correctly labeled the text, 0 otherwise) to calculate the phi-coefficient
labeled_correctly01 = [None]*np.size(testset['review'])

# Add a column 'prediction' which contains the model's predictions
prediction = [None]*np.size(testset['review'])

i = 0
for label1, label2 in zip(testset['label 1'], testset['label 2']):
  if(label1 == label2):
    equal[i] = True
    equal01[i] = 1
  else:
    equal[i] = False
    equal01[i] = 0
  i += 1
  
testset['equal'] = equal

In [ ]:
# Calculate the accuracy of the whole testset as well as 
# only the parts where the inital two labels are equal (accuracy_clear)
# or differ (accuracy_diff)

accuracy_clear = 0.
accuracy_diff = 0.
n_clear = 0
n_diff = 0
all_predicted_labels = []
predicted_labels_equal_initial_label = []
predicted_labels_diff_initial_label = []
true_label_equal = []
true_label_diff = []

# counter to go through all the reviews and fill the column 'labeled_correctly' with true if they have been labeled correctly, and false if not
i = 0

for label, review, equal in zip(testset['kano_labels'],  testset['review'], testset['equal']):
  
  test_prediction, raw_outputs = model.predict([review])
  
  predicted_label = ''

  if test_prediction[0] == 0:
    predicted_label = 'B'
    prediction[i] = 'B'
  if test_prediction[0] == 1:
    predicted_label = 'D'
    prediction[i] = 'D'
  if test_prediction[0] == 2:
    predicted_label = 'I'
    prediction[i] = 'I'
  if test_prediction[0] == 3:
    predicted_label = 'P'
    prediction[i] = 'P'

  all_predicted_labels.append(predicted_label)
  
  if equal:
    n_clear += 1
    predicted_labels_equal_initial_label.append(predicted_label)
    true_label_equal.append(label)
    if label == predicted_label:
      accuracy_clear += 1.

  if not equal:
    n_diff += 1
    predicted_labels_diff_initial_label.append(predicted_label)
    true_label_diff.append(label)
    if label == predicted_label:
      accuracy_diff += 1.

  if label == predicted_label:
    labeled_correctly01[i] = 1
  else:
    labeled_correctly01[i] = 0
  i+=1

testset['labeled_correctly'] = labeled_correctly01
testset['prediction'] = prediction

accuracy = (accuracy_clear + accuracy_diff) / (n_clear + n_diff)

accuracy_clear /= n_clear
accuracy_diff /= n_diff

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
testset.to_excel("Brunotte_with_predictions_test3_rembert.xlsx")  

In [ ]:
print('Accuracy Trainingskorpus:                                                  ', round(accuracy*100, 2), '%,  n =', (n_clear + n_diff))
print('Accuracy Trainingskorpus only on the data that coincides on initial labels:', round(accuracy_clear*100, 2), '%,  n =', n_clear)
print('Accuracy Trainingskorpus only on the data that differs on initial labels:  ', round(accuracy_diff*100, 2), '%,  n = ', n_diff)

Accuracy Trainingskorpus:                                                   71.52 %,  n = 1622
Accuracy Trainingskorpus only on the data that coincides on initial labels: 76.5 %,  n = 1383
Accuracy Trainingskorpus only on the data that differs on initial labels:   42.68 %,  n =  239


In [ ]:
# Calculate precision, recall and f1-score for each label

test_df_metrics = precision_recall_fscore_support(testset['kano_labels'], all_predicted_labels, average=None,labels=['B', 'D', 'I', 'P'])

print('Precision B: ', test_df_metrics[0][0])
print('Precision D: ', test_df_metrics[0][1])
print('Precision I: ', test_df_metrics[0][2])
print('Precision P: ', test_df_metrics[0][3])

print('Recall B: ', test_df_metrics[1][0])
print('Recall D: ', test_df_metrics[1][1])
print('Recall I: ', test_df_metrics[1][2])
print('Recall P: ', test_df_metrics[1][3])

print('F1-score B: ', test_df_metrics[2][0])
print('F1-score D: ', test_df_metrics[2][1])
print('F1-score I: ', test_df_metrics[2][2])
print('F1-score P: ', test_df_metrics[2][3])

Precision B:  0.9025695931477516
Precision D:  0.40119760479041916
Precision I:  0.5
Precision P:  0.4797687861271676
Recall B:  0.76497277676951
Recall D:  0.7052631578947368
Recall I:  0.03333333333333333
Recall P:  0.6303797468354431
F1-score B:  0.8280943025540275
F1-score D:  0.5114503816793894
F1-score I:  0.0625
F1-score P:  0.5448577680525165


In [ ]:
# Calculates the phi-coefficient of the labels being predicted correctly and the researchers being 

print('MCC for initial agreement of raters and correct or false labeling',matthews_corrcoef(labeled_correctly01, equal01))

MCC for initial agreement of raters and correct or false labeling 0.2656233453744388


## 10-fold Cross Validation on a combination of the Stanik dataset and the Brunotte dataset

In [ ]:
# Import combined downsampled dataset 
df_combined = pd.read_excel('DATASET_Trainingskorpus_combined_test3.xlsx')

df_combined = df_combined.drop(columns = ['Unnamed: 0'])

# Rename column so that simple transformers can handle it
df_combined = df_combined.rename(columns = {'review': 'text'})
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2972 non-null   object
 1   kano_labels  2972 non-null   object
 2   labels       2972 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.8+ KB


In [ ]:
# Show the distribution of the labels
print(df_combined['labels'].value_counts())

1    743
0    743
2    743
3    743
Name: labels, dtype: int64


In [ ]:
# 10-fold cross validation on the combined dataset

n2 = 10
kf2 = KFold(n_splits=n2, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc2 = []
resultsf12 = []
resultsPrecision2 = []
resultsRecall2 = []

model_args2 = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create a ClassificationModel
model2 = ClassificationModel(
    'rembert',
    'google/rembert',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf2.split(df_combined):
  train_df2 = df_combined.iloc[train_index]
  val_df2 = df_combined.iloc[val_index]

  model2.train_model(df_combined)
  result2, model_outputs2, wrong_predictions2 = model2.eval_model(val_df2, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)

  print(result2)
  # Add scores to the respective lists
  resultsAcc2.append(result2['acc'])
  resultsf12.append(result2['f1'])
  resultsPrecision2.append(result2['precision'])
  resultsRecall2.append(result2['recall'])

  print("results accuracy",resultsAcc2)
  print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
  print('results f1: ', resultsf12)
  print('results precision: ', resultsPrecision2)
  print('results recall: ', resultsRecall2)


Some weights of RemBertForSequenceClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.7259211140934692, 'f1': array([0.87951807, 0.80263158, 0.79710145, 0.67142857]), 'acc': 0.7919463087248322, 'recall': array([0.96052632, 0.85915493, 0.74324324, 0.61038961]), 'precision': array([0.81111111, 0.75308642, 0.859375  , 0.74603175]), 'eval_loss': 0.5778322972749409}
results accuracy [0.7919463087248322]
Mean-accuracy: 0.7919463087248322
results f1:  [array([0.87951807, 0.80263158, 0.79710145, 0.67142857])]
results precision:  [array([0.81111111, 0.75308642, 0.859375  , 0.74603175])]
results recall:  [array([0.96052632, 0.85915493, 0.74324324, 0.61038961])]


...

{'mcc': 0.9280479420984653, 'f1': array([0.97333333, 0.93181818, 0.95652174, 0.92810458]), 'acc': 0.9461279461279462, 'recall': array([1.        , 0.92134831, 0.98214286, 0.89873418]), 'precision': array([0.94805195, 0.94252874, 0.93220339, 0.95945946]), 'eval_loss': 0.234503846419485}
results accuracy [0.7919463087248322, 0.8120805369127517, 0.9191919191919192, 0.9158249158249159, 0.9090909090909091, 0.9562289562289562, 0.9629629629629629, 0.9595959595959596, 0.9461279461279462]
Mean-accuracy: 0.908116712740128
results f1:  [array([0.87951807, 0.80263158, 0.79710145, 0.67142857]), array([0.90588235, 0.76510067, 0.90647482, 0.65217391]), array([0.96256684, 0.87804878, 0.96598639, 0.84671533]), array([0.96296296, 0.88571429, 0.94339623, 0.85714286]), array([0.94827586, 0.91463415, 0.94666667, 0.84146341]), array([0.97058824, 0.95774648, 0.97777778, 0.91176471]), array([0.97260274, 0.95945946, 0.96402878, 0.95652174]), array([0.99337748, 0.94193548, 0.95238095, 0.95035461]), array([0.973

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.9463397235857958, 'f1': array([0.97810219, 0.96052632, 0.97333333, 0.92903226]), 'acc': 0.9595959595959596, 'recall': array([0.98529412, 0.97333333, 0.98648649, 0.9       ]), 'precision': array([0.97101449, 0.94805195, 0.96052632, 0.96      ]), 'eval_loss': 0.17641703706038625}
results accuracy [0.7919463087248322, 0.8120805369127517, 0.9191919191919192, 0.9158249158249159, 0.9090909090909091, 0.9562289562289562, 0.9629629629629629, 0.9595959595959596, 0.9461279461279462, 0.9595959595959596]
Mean-accuracy: 0.9132646374257112
results f1:  [array([0.87951807, 0.80263158, 0.79710145, 0.67142857]), array([0.90588235, 0.76510067, 0.90647482, 0.65217391]), array([0.96256684, 0.87804878, 0.96598639, 0.84671533]), array([0.96296296, 0.88571429, 0.94339623, 0.85714286]), array([0.94827586, 0.91463415, 0.94666667, 0.84146341]), array([0.97058824, 0.95774648, 0.97777778, 0.91176471]), array([0.97260274, 0.95945946, 0.96402878, 0.95652174]), array([0.99337748, 0.94193548, 0.95238095, 0.9

In [ ]:
print("results accuracy",resultsAcc2)
print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
print('results f1: ', resultsf12)
print('results precision: ', resultsPrecision2)
print('results recall: ', resultsRecall2)

results accuracy [0.7919463087248322, 0.8120805369127517, 0.9191919191919192, 0.9158249158249159, 0.9090909090909091, 0.9562289562289562, 0.9629629629629629, 0.9595959595959596, 0.9461279461279462, 0.9595959595959596]
Mean-accuracy: 0.8739278289278289
results f1:  [array([0.87951807, 0.80263158, 0.79710145, 0.67142857]), array([0.90588235, 0.76510067, 0.90647482, 0.65217391]), array([0.96256684, 0.87804878, 0.96598639, 0.84671533]), array([0.96296296, 0.88571429, 0.94339623, 0.85714286]), array([0.94827586, 0.91463415, 0.94666667, 0.84146341]), array([0.97058824, 0.95774648, 0.97777778, 0.91176471]), array([0.97260274, 0.95945946, 0.96402878, 0.95652174]), array([0.99337748, 0.94193548, 0.95238095, 0.95035461]), array([0.97333333, 0.93181818, 0.95652174, 0.92810458]), array([0.97810219, 0.96052632, 0.97333333, 0.92903226])]
results precision:  [array([0.81111111, 0.75308642, 0.859375  , 0.74603175]), array([0.84615385, 0.82608696, 0.94029851, 0.63380282]), array([0.9375    , 0.83076923

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

PrecisionRecallF1Table(resultsPrecision2, resultsRecall2, resultsf12)

,B,D,I,P
Average precision,0.936885,0.890432,0.952389,0.872838
Average recall,0.975237,0.912050,0.925992,0.839259
Average F1-Score,0.954721,0.899762,0.938367,0.854470
